In [30]:
import nltk
import pandas as pd
import json
import re
import os
import copy
nltk.download('book')

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/sh/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to /Users/sh/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to /Users/sh/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to /Users/sh/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /Users/sh/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /Users/sh/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /Users/sh/nltk_data...
[nltk_data]    |   Package

True

In [31]:
wine130_df = pd.read_csv('wine130.csv')
wine150_df = pd.read_csv('wine150.csv')

In [32]:
if os.path.isfile('./wine_df.json'):
    with open('wine_df.json', 'r') as json_file:
        wine_df = json.load(json_file)
else:
    wine_df = pd.concat([wine130_df['description'], wine150_df['description']]).reset_index(drop=True)
    wine_df = list(map(lambda x: x.replace('\n', ' '), wine_df))
    wine_df = list(map(lambda x: re.sub(r'[^ A-Za-z]', '', x), wine_df))
    wine_df = list(map(lambda x: nltk.tokenize.word_tokenize(x), wine_df))
    wine_df = list(map(lambda x: list(map(lambda y: y.lower(), x)), wine_df))
    wine_df = list(map(lambda x: nltk.tag.pos_tag(x), wine_df))
    
    json_value = json.dumps(wine_df)
    json_value = json.loads(json_value)
    with open('wine_df.json', 'w') as json_file:
        json.dump(json_value, json_file, indent=2)

wine_df

[[['aromas', 'NNS'],
  ['include', 'VBP'],
  ['tropical', 'JJ'],
  ['fruit', 'NN'],
  ['broom', 'NN'],
  ['brimstone', 'NN'],
  ['and', 'CC'],
  ['dried', 'VBD'],
  ['herb', 'NN'],
  ['the', 'DT'],
  ['palate', 'NN'],
  ['isnt', 'NN'],
  ['overly', 'RB'],
  ['expressive', 'JJ'],
  ['offering', 'NN'],
  ['unripened', 'JJ'],
  ['apple', 'NN'],
  ['citrus', 'NN'],
  ['and', 'CC'],
  ['dried', 'JJ'],
  ['sage', 'NN'],
  ['alongside', 'RB'],
  ['brisk', 'JJ'],
  ['acidity', 'NN']],
 [['this', 'DT'],
  ['is', 'VBZ'],
  ['ripe', 'JJ'],
  ['and', 'CC'],
  ['fruity', 'NN'],
  ['a', 'DT'],
  ['wine', 'NN'],
  ['that', 'WDT'],
  ['is', 'VBZ'],
  ['smooth', 'VBN'],
  ['while', 'IN'],
  ['still', 'RB'],
  ['structured', 'VBN'],
  ['firm', 'NN'],
  ['tannins', 'NNS'],
  ['are', 'VBP'],
  ['filled', 'VBN'],
  ['out', 'RP'],
  ['with', 'IN'],
  ['juicy', 'NN'],
  ['red', 'JJ'],
  ['berry', 'NN'],
  ['fruits', 'NNS'],
  ['and', 'CC'],
  ['freshened', 'VBN'],
  ['with', 'IN'],
  ['acidity', 'NN'],
  ['i

In [33]:
cnt = {}
for i in range(len(wine_df)):
  for j in range(len(wine_df[i])):
    if wine_df[i][j][0] in cnt:
      cnt[wine_df[i][j][0]] += 1
    else:
      cnt[wine_df[i][j][0]] = 1
words_count = list(cnt.items())
words_count = sorted(words_count, key=lambda x: -x[1])
print(words_count)

[('and', 752359), ('the', 479430), ('a', 395134), ('of', 357086), ('with', 273067), ('this', 223740), ('is', 208249), ('wine', 165671), ('flavors', 140517), ('in', 136944), ('to', 119541), ('it', 119395), ('its', 108812), ('fruit', 101524), ('on', 91378), ('that', 82231), ('but', 79486), ('aromas', 75443), ('finish', 72666), ('acidity', 67562), ('palate', 67028), ('tannins', 63041), ('from', 61839), ('cherry', 58066), ('ripe', 53712), ('drink', 53601), ('are', 52790), ('for', 52139), ('has', 51996), ('black', 50006), ('by', 47137), ('spice', 41441), ('an', 40650), ('as', 39578), ('notes', 38625), ('now', 38474), ('rich', 38447), ('dry', 38425), ('red', 38072), ('oak', 36328), ('fresh', 34623), ('sweet', 33687), ('nose', 32787), ('berry', 32592), ('soft', 31286), ('blackberry', 27681), ('plum', 27404), ('shows', 27282), ('crisp', 26753), ('blend', 26743), ('good', 26736), ('very', 26422), ('fruits', 25763), ('apple', 24288), ('white', 24167), ('vanilla', 24008), ('at', 23868), ('more', 

In [34]:
word2event = {
    'fruit': 'F',
    'fruity': 'F',
    'aromas': 'A',
    'apple': 'AP',
    'berry': 'B',
    'blackberry': 'B',
    'raspberry': 'B',
    'cherry': 'C',
    'tannins': 'T',
    'acidity': 'AC',
    'oak': 'O',
    'cabernet': 'CA',
    'spice': 'S',
    'plum': 'P',
    'ripe': 'RR',
    'blend': 'BL',
    'note': 'N',
    'notes': 'N',
    'dry': 'D',
    'sweet': 'SW',
    'fresh': 'F',
    'soft': 'SO',
    'vanilla': 'V',
    'chocolate': 'CH',
}
'''
'pepper': 'PE',
'peach': "PEA"
'''

'\n\'pepper\': \'PE\',\n\'peach\': "PEA"\n'

In [35]:
filtered = list(map(lambda x: list(filter(lambda y: y[0] in word2event, x)), wine_df))
filtered = list(filter(lambda x: len(x) >= 8 , filtered))
filtered = list(map(lambda x: x[:8], filtered))
filtered = list(map(lambda x: list(map(lambda y: '#' + word2event[y[0]], x)), filtered))
# filtered = list(map(lambda x: {'es': x, 'order': -1}, filtered))
print(len(filtered))

7079


In [36]:
AoI = {}
for i in range(len(filtered)):
    for j in range(len(filtered[i]) - 1):
        key = filtered[i][j] + filtered[i][j+1] + str(j) + str(j + 1)
        if key not in AoI:
            AoI[key] = [0, [j, j + 1], [filtered[i][j], filtered[i][j + 1]]]
        AoI[key][0] += 1
AoI = list(AoI.items())
AoI = sorted(AoI, key=lambda x: -x[1][0])
print(AoI)

[('#BL#CA01', [715, [0, 1], ['#BL', '#CA']]), ('#A#B01', [364, [0, 1], ['#A', '#B']]), ('#CA#CA12', [293, [1, 2], ['#CA', '#CA']]), ('#C#B45', [291, [4, 5], ['#C', '#B']]), ('#A#RR01', [290, [0, 1], ['#A', '#RR']]), ('#C#B34', [287, [3, 4], ['#C', '#B']]), ('#F#AC67', [268, [6, 7], ['#F', '#AC']]), ('#B#A12', [265, [1, 2], ['#B', '#A']]), ('#RR#B01', [242, [0, 1], ['#RR', '#B']]), ('#B#A01', [212, [0, 1], ['#B', '#A']]), ('#CA#A12', [211, [1, 2], ['#CA', '#A']]), ('#B#C23', [200, [2, 3], ['#B', '#C']]), ('#RR#B12', [199, [1, 2], ['#RR', '#B']]), ('#A#P01', [199, [0, 1], ['#A', '#P']]), ('#B#C34', [190, [3, 4], ['#B', '#C']]), ('#AC#T67', [186, [6, 7], ['#AC', '#T']]), ('#B#C12', [186, [1, 2], ['#B', '#C']]), ('#T#F67', [180, [6, 7], ['#T', '#F']]), ('#C#B56', [179, [5, 6], ['#C', '#B']]), ('#T#F56', [178, [5, 6], ['#T', '#F']]), ('#A#B12', [177, [1, 2], ['#A', '#B']]), ('#B#F45', [175, [4, 5], ['#B', '#F']]), ('#RR#F12', [169, [1, 2], ['#RR', '#F']]), ('#RR#F01', [168, [0, 1], ['#RR', 

In [37]:
AoI = AoI[:50]

In [38]:
color = [
    "#003f5c", "#2f4b7c", "#665191", "#a05195", "#d45087",
    "#f95d6a", "#ff7c43", "#ffa600", "#488f31", "#28b4c8",
    "#a2c8ec", "#FFD700", "#87CEEB", "#FFB6C1", "#20B2AA",
    "#778899", "#32CD32", "#6A5ACD", "#ff6347", "#90ee90"
]
event2color = {}
cnt = 0
for i in range(len(AoI)):
    if AoI[i][1][2][0] not in event2color:
        event2color[AoI[i][1][2][0]] = color[cnt]
        cnt += 1
    if AoI[i][1][2][1] not in event2color:
        event2color[AoI[i][1][2][1]] = color[cnt]
        cnt += 1

In [39]:
AoI[0][1]

[715, [0, 1], ['#BL', '#CA']]

In [40]:
edges = [[] for _ in range(len(AoI))]
for i in range(len(AoI)):
    for j in range(len(AoI)):
        if i == j:
            continue
        if not set(AoI[i][1][1]) & set(AoI[j][1][1]):
            edges[i].append(j)
            edges[j].append(i)

In [41]:
cnt = 0
nodes = [-1 for _ in range(len(AoI))]
disjoint = [[] for _ in range(len(AoI))]
ds = [[] for _ in range(len(AoI))]
def dfs(curr):
    global cnt, nodes

    adj_colors = []
    for adj in edges[curr]:
        if nodes[adj] != -1:
            adj_colors.append(nodes[adj])
    
    for i in range(len(AoI)):
        if i not in adj_colors and not set(AoI[curr][1][1]) & set(disjoint[i]):
            nodes[curr] = i
            disjoint[i].extend(AoI[curr][1][1])
            ds[i].append(curr)
            break

    for next in edges[curr]:
        if nodes[next] == -1:
            dfs(next)

dfs(0)

nodes

[0,
 2,
 4,
 1,
 6,
 3,
 5,
 10,
 8,
 12,
 14,
 7,
 16,
 18,
 9,
 11,
 20,
 13,
 15,
 17,
 22,
 19,
 24,
 26,
 28,
 30,
 27,
 21,
 29,
 31,
 32,
 23,
 25,
 34,
 36,
 38,
 33,
 35,
 37,
 40,
 39,
 43,
 41,
 42,
 46,
 45,
 44,
 47,
 48,
 49]

In [42]:
board = []
for i in range(max(nodes) + 1):
    offset = len(board)
    max_height = 0
    for node in ds[i]:
        max_height = max(max_height, AoI[node][1][0])
    for j in range(max_height):
        board.append(['#ffffff' for _ in range(len(filtered[0]))])
    for node in ds[i]:
        for j in range(AoI[node][1][0]):
            board[offset + j][AoI[node][1][1][0]] = event2color[AoI[node][1][2][0]]
            board[offset + j][AoI[node][1][1][1]] = event2color[AoI[node][1][2][1]]
                    

[['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '#d45087',
  '#a05195',
  '#ff7c43',
  '#ffa600'],
 ['#003f5c',
  '#2f4b7c',
  '#a05195',
  '#d45087',
  '

In [43]:
label = []
for key in word2event:
    if key not in event2color: continue
    label.append([key, event2color['#' + word2event[key]]])
lst = [label, board]
json_value = json.dumps(lst)
json_value = json.loads(json_value)
with open('wine.json', 'w') as json_file:
    json.dump(json_value, json_file, indent=2)